In [16]:
import torch
torch.cuda.is_available()
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
import torch

from transformations.transform import WaveformToInput as TorchTransform
from architecture.backend.yamnet.params import YAMNetParams
from architecture.backend.yamnet.model import yamnet
from architecture.backend.yamnet.model import yamnet_category_metadata

from architecture.classifier.classification import Classifier

from datasets.SvdExDataset import SvdCutOffShort
from tqdm import tqdm
import IPython

from datasets.SvdExDataset import SvdExtendedVoiceDataset


%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import s3prl.hub

[]
{'data': tensor([[[12.3761, 11.7492, 11.3360,  ...,  7.6452,  7.6120,  7.9758],
         [12.2741, 11.4850, 11.3448,  ...,  6.8388,  6.5159,  6.7437],
         [12.2936, 11.7842,  9.7161,  ...,  6.7731,  5.9573,  6.6129],
         ...,
         [12.4582, 12.3116, 11.1653,  ...,  6.7223,  7.0045,  7.2922],
         [12.2054, 11.9077, 11.1359,  ...,  6.9165,  7.2385,  7.2134],
         [12.3611, 12.1957, 11.9286,  ...,  6.7778,  6.3635,  6.1738]]]), 'sampling_rate': 50000, 'classification': True}


In [40]:
# tera = s3prl.hub.tera().to(device="cuda:0")
wav2vec2 = s3prl.hub.wav2vec2(refresh=True).to(device="cuda:0")

In [44]:
distilhubert = s3prl.hub.distilhubert(refresh=True).to(device="cuda:0")

[UpstreamExpert] - Using the default upstream expert config
[DistillerModel] - Expands the output dimension by 3 times
[DistillerModel] - Pred layers: [4, 8, 12]
[TransformerEncoder] - Attention type = original
[DistillerModel] - Out layer type: expand-last
[DistillerModel] - Inter dim = 768


In [37]:
hp = {}
hp["augmentations"]=[]
dataset = SvdExtendedVoiceDataset(r"/home/yiftach.ede@staff.technion.ac.il/data/SVD/",hp=hp,classification_binary=True)

for data in tqdm(dataset):
    data["data"]=data["data"].to(device="cuda:0")
    out=tera(data["data"])
    # break


out['last_hidden_state'].shape

[]


100%|██████████| 23234/23234 [01:52<00:00, 207.36it/s]


torch.Size([1, 438, 768])

In [50]:
hp = {}
hp["augmentations"]=[]
dataset = SvdExtendedVoiceDataset(r"/home/yiftach.ede@staff.technion.ac.il/data/SVD/",hp=hp,classification_binary=True)

for data in tqdm(dataset):
    data["data"]=data["data"].to(device="cuda:0")
    out=wav2vec2(data["data"])
    break


out.keys()

[]


  0%|          | 0/23234 [00:00<?, ?it/s]


dict_keys(['_hidden_states_info', 'hidden_states', 'last_hidden_state', 'hidden_state_0', 'hidden_state_1', 'hidden_state_2', 'hidden_state_3', 'hidden_state_4', 'hidden_state_5', 'hidden_state_6', 'hidden_state_7', 'hidden_state_8', 'hidden_state_9', 'hidden_state_10', 'hidden_state_11'])

In [49]:
hp = {}
hp["augmentations"]=[]
dataset = SvdExtendedVoiceDataset(r"/home/yiftach.ede@staff.technion.ac.il/data/SVD/",hp=hp,classification_binary=True)

for data in tqdm(dataset):
    data["data"]=data["data"].to(device="cuda:0")
    out=distilhubert(data["data"])
    break


out.keys()

[]


  0%|          | 0/23234 [00:00<?, ?it/s]


dict_keys(['last_hidden_state', 'hidden_states', 'pad_mask', 'paper'])